In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
# Gerekli kütüphaneleri içe aktarma
import numpy as np
import cv2
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical

import kerastuner as kt


<ipython-input-4-eac48f44ad09>:13: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [5]:
import os
print(os.listdir("/gdrive/My Drive/Banana_Ripeness_Classification"))


['unripe', 'ripe', 'rotten', 'overripe']


In [6]:
DATADIR = '/gdrive/My Drive/Banana_Ripeness_Classification/'
CATEGORIES = ["overripe", "ripe", "rotten", "unripe"]
IMG_SIZE = 100


In [7]:
def create_all_data():
    all_data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                all_data.append([new_array, class_num])
            except Exception as e:
                pass
    return all_data

In [8]:
all_data = create_all_data()
print("Total number of images:", len(all_data))

Total number of images: 13478


In [9]:

X = []
y = []
for features, label in all_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)


In [11]:
# Etiketleri kategorik hale dönüştürme
train_yCl = to_categorical(y_train, num_classes=4)
test_yCl = to_categorical(y_test, num_classes=4)
valid_yCl = to_categorical(y_val, num_classes=4)

In [12]:
print(y_test)

[1 1 3 ... 2 1 1]


In [13]:
def build_model(hp):
    model = Sequential()

    # İlk Conv2D katmanındaki birim sayısını ayarlama
    model.add(Conv2D(hp.Int('conv_1_units', min_value=32, max_value=128, step=32),
                     (3, 3), padding='same',
                     input_shape=(IMG_SIZE, IMG_SIZE, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

   # İkinci Conv2D katmanındaki birim sayısını ayarlama
    model.add(Conv2D(hp.Int('conv_2_units', min_value=32, max_value=128, step=32),
                     (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    # Dense katmanındaki nöron sayısını ayarlama
    model.add(Dense(hp.Int('dense_units', min_value=64, max_value=256, step=64), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [14]:
# Hiperparametre ayarlayıcısını başlatma
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='banana_classification'
)



In [15]:
print(y_test)

[1 1 3 ... 2 1 1]


In [16]:
# Callbacks tanımlamak
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10)
]

# Hiperparametre ayarlamasını başlatma
tuner.search(x_train, train_yCl, batch_size=32, epochs=10, validation_data=(x_val, valid_yCl), callbacks=callbacks)


Trial 5 Complete [00h 02m 42s]
val_accuracy: 0.8241525292396545

Best val_accuracy So Far: 0.8241525292396545
Total elapsed time: 00h 33m 48s


In [17]:
# En iyi hiperparametreleri elde etme
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Modeli en iyi hiperparametrelerle oluşturma ve eğitme
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, train_yCl, epochs=50, validation_data=(x_val, valid_yCl))


Epoch 1/50
266/266 [==============================] - 17s 60ms/step - loss: 8.1987 - accuracy: 0.3322 - val_loss: 1.3614 - val_accuracy: 0.3263
Epoch 2/50
266/266 [==============================] - 16s 59ms/step - loss: 1.3323 - accuracy: 0.3689 - val_loss: 1.2964 - val_accuracy: 0.3888
Epoch 3/50
266/266 [==============================] - 16s 60ms/step - loss: 1.2829 - accuracy: 0.4118 - val_loss: 1.2474 - val_accuracy: 0.4354
Epoch 4/50
266/266 [==============================] - 16s 60ms/step - loss: 1.1715 - accuracy: 0.4810 - val_loss: 1.1931 - val_accuracy: 0.4502
Epoch 5/50
266/266 [==============================] - 16s 60ms/step - loss: 1.0979 - accuracy: 0.5226 - val_loss: 1.1047 - val_accuracy: 0.5148
Epoch 6/50
266/266 [==============================] - 16s 59ms/step - loss: 0.9798 - accuracy: 0.5807 - val_loss: 0.9385 - val_accuracy: 0.5985
Epoch 7/50
266/266 [==============================] - 16s 61ms/step - loss: 0.8190 - accuracy: 0.6676 - val_loss: 0.6746 - val_accuracy:

In [18]:
# Modeli değerlendirmek
print("Training Accuracy:", model.evaluate(x_train, train_yCl)[1])
print("Validation Accuracy:", model.evaluate(x_val, valid_yCl)[1])
print("Test Accuracy:", model.evaluate(x_test, test_yCl)[1])


266/266 [==============================] - 4s 14ms/step - loss: 0.0241 - accuracy: 0.9936
Training Accuracy: 0.9936395883560181
30/30 [==============================] - 0s 13ms/step - loss: 0.5751 - accuracy: 0.8856
Validation Accuracy: 0.8855932354927063
127/127 [==============================] - 2s 13ms/step - loss: 0.7797 - accuracy: 0.8746
Test Accuracy: 0.8746290802955627


In [19]:
# Test seti üzerindeki tahminler
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = y_test

127/127 [==============================] - 2s 14ms/step


In [20]:
# Performans metrikleri
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred)


In [21]:
# Sonuçları görüntüleme
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

Accuracy: 0.8746290801186943
Precision: 0.8777836271460271
Recall: 0.8746290801186943
F1 Score: 0.8747896604593862

Confusion Matrix:
 [[ 715   46   61    0]
 [  81  977  128    7]
 [  54   44 1239   21]
 [   2    9   54  606]]

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.87      0.85       822
           1       0.91      0.82      0.86      1193
           2       0.84      0.91      0.87      1358
           3       0.96      0.90      0.93       671

    accuracy                           0.87      4044
   macro avg       0.88      0.88      0.88      4044
weighted avg       0.88      0.87      0.87      4044

